In [ ]:
"""
PMCC Optimization Framework
==========================

Systematic framework for optimizing Poor Man's Covered Call strategy
to maximize Sharpe ratio through various enhancement techniques:

1. Market Timing Filters (VIX, trend, regime)
2. Delta Target Optimization  
3. Greeks-Based Management
4. IV Rank Integration
5. Collar Modifications

This notebook provides a parameterized backtesting engine specifically
designed for PMCC optimization research.
"""

In [ ]:
import sys
sys.path.append('/Users/nish_macbook/trading/daily-optionslab')

import pandas as pd
import numpy as np
import warnings
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Import our existing infrastructure
from optionslab.data_loader import load_data

warnings.filterwarnings('ignore')
print("📊 PMCC Optimization Framework Initialized")

In [ ]:
class PMCCOptimizer:
    """
    Parameterized PMCC backtest engine for systematic optimization
    """
    
    def __init__(self, data, spy_data, initial_capital=10000):
        """
        Initialize the PMCC optimizer
        
        Args:
            data: Options data DataFrame
            spy_data: SPY price data for market filters
            initial_capital: Starting capital for backtest
        """
        self.data = data
        self.spy_data = spy_data
        self.initial_capital = initial_capital
        
        # Add technical indicators to SPY data
        self._add_technical_indicators()
        
        # Default strategy parameters (baseline)
        self.default_params = {
            # LEAP Selection
            'leap_dte_min': 600,
            'leap_dte_max': 800,
            'leap_delta_min': 0.70,
            'leap_delta_max': 0.85,
            
            # Short Call Selection  
            'short_dte_min': 30,
            'short_dte_max': 45,
            'short_delta_min': 0.20,
            'short_delta_max': 0.30,
            
            # Rolling Rules
            'roll_profit_pct': 50,      # Roll at 50% profit
            'roll_dte_threshold': 21,   # Roll at 21 DTE
            'leap_roll_dte': 120,       # Roll LEAP at 120 DTE
            
            # Market Timing Filters (OFF by default)
            'use_vix_filter': False,
            'vix_entry_max': 25,        # Enter when VIX < 25
            'vix_exit_min': 35,         # Exit when VIX > 35
            
            'use_trend_filter': False,
            'sma_period': 50,           # 50-day moving average
            'require_uptrend': True,    # Only trade in uptrend
            
            'use_rsi_filter': False,
            'rsi_period': 14,
            'rsi_min': 30,              # Avoid oversold
            'rsi_max': 70,              # Avoid overbought
            
            # IV Filters (OFF by default)
            'use_iv_filter': False,
            'iv_rank_min': 0.20,        # Min IV rank to sell calls
            'iv_rank_max': 0.80,        # Max IV rank to buy LEAPs
            
            # Position Management
            'max_positions': 1,
            'position_size_pct': 0.90,  # Use 90% of capital
            
            # Execution
            'commission_per_contract': 0.65,
            'slippage_pct': 0.005       # 0.5% slippage
        }
    
    def _add_technical_indicators(self):
        """Add technical indicators to SPY data"""
        # Simple moving averages
        self.spy_data['sma_20'] = self.spy_data['close'].rolling(20).mean()
        self.spy_data['sma_50'] = self.spy_data['close'].rolling(50).mean()
        self.spy_data['sma_200'] = self.spy_data['close'].rolling(200).mean()
        
        # RSI
        delta = self.spy_data['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
        rs = gain / loss
        self.spy_data['rsi'] = 100 - (100 / (1 + rs))
        
        # VIX proxy (simplified volatility measure)
        self.spy_data['returns'] = self.spy_data['close'].pct_change()
        self.spy_data['vol_20'] = self.spy_data['returns'].rolling(20).std() * np.sqrt(252) * 100
        
        print(f"✅ Added technical indicators to SPY data")
    
    def run_backtest(self, params=None, start_date='2023-01-01', end_date='2024-12-31', verbose=False):
        """
        Run PMCC backtest with given parameters
        
        Args:
            params: Strategy parameters (uses defaults if None)
            start_date: Backtest start date
            end_date: Backtest end date  
            verbose: Print detailed logs
            
        Returns:
            Dictionary with backtest results
        """
        if params is None:
            params = self.default_params.copy()
        
        if verbose:
            print(f"🚀 Running PMCC backtest: {start_date} to {end_date}")
            print(f"📊 Parameters: {params}")
        
        # Filter data to date range
        test_data = self.data[
            (pd.to_datetime(self.data['date']) >= start_date) & 
            (pd.to_datetime(self.data['date']) <= end_date)
        ].copy()
        
        test_spy = self.spy_data[
            (pd.to_datetime(self.spy_data['date']) >= start_date) & 
            (pd.to_datetime(self.spy_data['date']) <= end_date)
        ].copy()
        
        if test_data.empty:
            return None
        
        # Initialize backtest state
        cash = self.initial_capital
        positions = []  # [{'type': 'leap'/'short', 'position': {...}, 'entry_date': ...}]
        equity_curve = []
        trades = []
        
        # Get unique trading dates
        trading_dates = sorted(test_data['date'].unique())
        
        for current_date in trading_dates:
            if verbose and len(equity_curve) % 50 == 0:
                print(f"📅 Processing {current_date} ({len(equity_curve)}/{len(trading_dates)})")
            
            # Get current market data
            daily_options = test_data[test_data['date'] == current_date]
            spy_info = test_spy[test_spy['date'] == current_date]
            
            if daily_options.empty or spy_info.empty:
                continue
                
            spy_price = spy_info['close'].iloc[0]
            
            # Apply market filters
            if not self._check_market_filters(spy_info.iloc[0], params):
                if verbose:
                    print(f"⚠️ Market filters failed for {current_date}")
                # Still track equity but don't trade
                equity_curve.append({
                    'date': current_date,
                    'cash': cash,
                    'positions_value': self._calculate_positions_value(positions, daily_options),
                    'total_value': cash + self._calculate_positions_value(positions, daily_options),
                    'spy_price': spy_price
                })
                continue
            
            # Check for position exits and rolls
            positions, cash, new_trades = self._process_exits_and_rolls(
                positions, daily_options, current_date, cash, params, verbose
            )
            trades.extend(new_trades)
            
            # Check for new entries
            if len([p for p in positions if p['type'] == 'leap']) < params['max_positions']:
                new_positions, cash, new_trades = self._process_entries(
                    daily_options, current_date, cash, params, verbose
                )
                positions.extend(new_positions)
                trades.extend(new_trades)
            
            # Record equity curve
            positions_value = self._calculate_positions_value(positions, daily_options)
            equity_curve.append({
                'date': current_date,
                'cash': cash,
                'positions_value': positions_value,
                'total_value': cash + positions_value,
                'spy_price': spy_price,
                'leap_positions': len([p for p in positions if p['type'] == 'leap']),
                'short_positions': len([p for p in positions if p['type'] == 'short'])
            })
        
        # Calculate final metrics
        results = self._calculate_results(equity_curve, trades, params, verbose)
        results['params'] = params
        results['trades'] = trades
        results['equity_curve'] = equity_curve
        
        return results
    
    def _check_market_filters(self, spy_info, params):
        """Check if market conditions allow trading"""
        
        # VIX filter (using volatility proxy)
        if params['use_vix_filter']:
            if 'vol_20' in spy_info and not pd.isna(spy_info['vol_20']):
                if spy_info['vol_20'] > params['vix_exit_min']:
                    return False  # Too volatile
        
        # Trend filter
        if params['use_trend_filter']:
            sma_col = f"sma_{params['sma_period']}"
            if sma_col in spy_info and not pd.isna(spy_info[sma_col]):
                if params['require_uptrend']:
                    if spy_info['close'] < spy_info[sma_col]:
                        return False  # Below trend
                else:
                    if spy_info['close'] > spy_info[sma_col]:
                        return False  # Above trend
        
        # RSI filter
        if params['use_rsi_filter']:
            if 'rsi' in spy_info and not pd.isna(spy_info['rsi']):
                if spy_info['rsi'] < params['rsi_min'] or spy_info['rsi'] > params['rsi_max']:
                    return False  # RSI out of range
        
        return True  # All filters passed
    
    def _process_entries(self, daily_options, current_date, cash, params, verbose):
        """Process new PMCC entries"""
        new_positions = []
        new_trades = []
        
        # Find suitable LEAP
        leap_option = self._find_leap_option(daily_options, params)
        if leap_option is None:
            return new_positions, cash, new_trades
        
        # Calculate LEAP cost
        leap_cost = leap_option['close'] * 100 + params['commission_per_contract']
        leap_cost *= (1 + params['slippage_pct'])  # Add slippage
        
        if leap_cost > cash * params['position_size_pct']:
            return new_positions, cash, new_trades  # Not enough capital
        
        # Buy LEAP
        cash -= leap_cost
        leap_position = {
            'type': 'leap',
            'entry_date': current_date,
            'strike': leap_option['strike'],
            'expiration': leap_option['expiration'],
            'entry_price': leap_option['close'],
            'cost': leap_cost,
            'delta': leap_option.get('delta', 0.8),
            'contracts': 1
        }
        new_positions.append(leap_position)
        
        new_trades.append({
            'date': current_date,
            'action': 'buy_leap',
            'strike': leap_option['strike'],
            'expiration': leap_option['expiration'],
            'price': leap_option['close'],
            'cost': leap_cost,
            'pnl': 0,
            'reason': 'new_pmcc_entry'
        })
        
        # Find suitable short call
        short_option = self._find_short_call(daily_options, leap_option, params)
        if short_option is not None:
            # Sell short call
            short_premium = short_option['close'] * 100 - params['commission_per_contract']
            short_premium *= (1 - params['slippage_pct'])  # Reduce for slippage
            
            cash += short_premium
            short_position = {
                'type': 'short',
                'entry_date': current_date,
                'strike': short_option['strike'],
                'expiration': short_option['expiration'],
                'entry_price': short_option['close'],
                'premium_received': short_premium,
                'delta': short_option.get('delta', 0.25),
                'contracts': 1,
                'leap_strike': leap_option['strike'],  # Link to LEAP
                'leap_expiration': leap_option['expiration']
            }
            new_positions.append(short_position)
            
            new_trades.append({
                'date': current_date,
                'action': 'sell_call',
                'strike': short_option['strike'],
                'expiration': short_option['expiration'],
                'price': short_option['close'],
                'cost': -short_premium,  # Negative = received premium
                'pnl': 0,
                'reason': 'pmcc_short_leg'
            })
        
        if verbose:
            print(f"✅ New PMCC entry: LEAP ${leap_option['strike']:.0f}, Short ${short_option['strike']:.0f if short_option is not None else 'None'}")
        
        return new_positions, cash, new_trades
    
    def _process_exits_and_rolls(self, positions, daily_options, current_date, cash, params, verbose):
        """Process position exits and rolls"""
        remaining_positions = []
        new_trades = []
        
        for position in positions:
            # Find current option data
            current_option = daily_options[
                (daily_options['strike'] == position['strike']) &
                (daily_options['expiration'] == position['expiration'])
            ]
            
            if current_option.empty:
                # Option may have expired or delisted
                if position['type'] == 'leap':
                    # LEAP expired - likely worthless
                    new_trades.append({
                        'date': current_date,
                        'action': 'leap_expired',
                        'strike': position['strike'],
                        'expiration': position['expiration'],
                        'price': 0,
                        'cost': 0,
                        'pnl': -position['cost'],
                        'reason': 'expiration'
                    })
                else:
                    # Short call expired - keep premium
                    new_trades.append({
                        'date': current_date,
                        'action': 'short_expired',
                        'strike': position['strike'],
                        'expiration': position['expiration'],
                        'price': 0,
                        'cost': 0,
                        'pnl': position['premium_received'],
                        'reason': 'expiration'
                    })
                continue  # Don't add to remaining positions
            
            current_price = current_option.iloc[0]['close']
            days_to_expiry = (pd.to_datetime(position['expiration']) - pd.to_datetime(current_date)).days
            
            # Check exit conditions
            should_exit, exit_reason = self._check_exit_conditions(
                position, current_price, days_to_expiry, params
            )
            
            if should_exit:
                # Execute exit
                if position['type'] == 'leap':
                    # Sell LEAP
                    proceeds = current_price * 100 - params['commission_per_contract']
                    proceeds *= (1 - params['slippage_pct'])
                    cash += proceeds
                    pnl = proceeds - position['cost']
                    
                    new_trades.append({
                        'date': current_date,
                        'action': 'sell_leap',
                        'strike': position['strike'],
                        'expiration': position['expiration'],
                        'price': current_price,
                        'cost': proceeds,
                        'pnl': pnl,
                        'reason': exit_reason
                    })
                else:
                    # Buy back short call
                    cost = current_price * 100 + params['commission_per_contract']
                    cost *= (1 + params['slippage_pct'])
                    cash -= cost
                    pnl = position['premium_received'] - cost
                    
                    new_trades.append({
                        'date': current_date,
                        'action': 'buy_call',
                        'strike': position['strike'],
                        'expiration': position['expiration'],
                        'price': current_price,
                        'cost': cost,
                        'pnl': pnl,
                        'reason': exit_reason
                    })
                
                if verbose:
                    print(f"🔄 Exit {position['type']}: ${position['strike']:.0f} @ ${current_price:.2f}, P&L: ${pnl:.2f} ({exit_reason})")
            
            else:
                # Keep position
                remaining_positions.append(position)
        
        return remaining_positions, cash, new_trades
    
    def _find_leap_option(self, daily_options, params):
        """Find suitable LEAP option"""
        calls = daily_options[daily_options['right'] == 'C'].copy()
        
        # Filter by DTE
        calls['dte'] = (pd.to_datetime(calls['expiration']) - pd.to_datetime(calls['date'])).dt.days
        leap_candidates = calls[
            (calls['dte'] >= params['leap_dte_min']) &
            (calls['dte'] <= params['leap_dte_max'])
        ]
        
        if leap_candidates.empty:
            return None
        
        # Filter by delta if available
        if 'delta' in leap_candidates.columns:
            leap_candidates = leap_candidates[
                (leap_candidates['delta'] >= params['leap_delta_min']) &
                (leap_candidates['delta'] <= params['leap_delta_max'])
            ]
        
        # Basic liquidity filter
        leap_candidates = leap_candidates[
            (leap_candidates['bid'] > 0) &
            (leap_candidates['ask'] > leap_candidates['bid']) &
            (leap_candidates['volume'] > 0)
        ]
        
        if leap_candidates.empty:
            return None
        
        # Select closest to target delta (0.80)
        if 'delta' in leap_candidates.columns:
            leap_candidates['delta_diff'] = abs(leap_candidates['delta'] - 0.80)
            best_leap = leap_candidates.nsmallest(1, 'delta_diff').iloc[0]
        else:
            # Fallback: select closest to ATM with longest DTE
            underlying_price = daily_options['underlying_price'].iloc[0]
            leap_candidates['strike_diff'] = abs(leap_candidates['strike'] - underlying_price * 0.85)
            best_leap = leap_candidates.nsmallest(1, 'strike_diff').iloc[0]
        
        return best_leap
    
    def _find_short_call(self, daily_options, leap_option, params):
        """Find suitable short call to sell against LEAP"""
        calls = daily_options[daily_options['right'] == 'C'].copy()
        
        # Filter by DTE
        calls['dte'] = (pd.to_datetime(calls['expiration']) - pd.to_datetime(calls['date'])).dt.days
        short_candidates = calls[
            (calls['dte'] >= params['short_dte_min']) &
            (calls['dte'] <= params['short_dte_max'])
        ]
        
        if short_candidates.empty:
            return None
        
        # Filter by delta if available
        if 'delta' in short_candidates.columns:
            short_candidates = short_candidates[
                (short_candidates['delta'] >= params['short_delta_min']) &
                (short_candidates['delta'] <= params['short_delta_max'])
            ]
        
        # Must be above LEAP strike
        short_candidates = short_candidates[
            short_candidates['strike'] > leap_option['strike']
        ]
        
        # Basic liquidity filter
        short_candidates = short_candidates[
            (short_candidates['bid'] > 0) &
            (short_candidates['ask'] > short_candidates['bid']) &
            (short_candidates['volume'] > 0)
        ]
        
        if short_candidates.empty:
            return None
        
        # Select closest to target delta (0.25)
        if 'delta' in short_candidates.columns:
            short_candidates['delta_diff'] = abs(short_candidates['delta'] - 0.25)
            best_short = short_candidates.nsmallest(1, 'delta_diff').iloc[0]
        else:
            # Fallback: select strike ~5% OTM
            underlying_price = daily_options['underlying_price'].iloc[0]
            target_strike = underlying_price * 1.05
            short_candidates['strike_diff'] = abs(short_candidates['strike'] - target_strike)
            best_short = short_candidates.nsmallest(1, 'strike_diff').iloc[0]
        
        return best_short
    
    def _check_exit_conditions(self, position, current_price, days_to_expiry, params):
        """Check if position should be exited"""
        
        if position['type'] == 'leap':
            # LEAP exit conditions
            if days_to_expiry <= params['leap_roll_dte']:
                return True, f'roll_dte_{days_to_expiry}'
                
        else:
            # Short call exit conditions
            # Profit target
            cost_to_close = current_price * 100
            profit = position['premium_received'] - cost_to_close
            profit_pct = (profit / position['premium_received']) * 100
            
            if profit_pct >= params['roll_profit_pct']:
                return True, f'profit_target_{profit_pct:.1f}%'
            
            # Time-based roll
            if days_to_expiry <= params['roll_dte_threshold']:
                return True, f'roll_dte_{days_to_expiry}'
                
            # Assignment risk (deep ITM)
            underlying_price = current_price  # Simplified - should get underlying
            if underlying_price > position['strike'] * 1.05:  # 5% ITM
                return True, 'assignment_risk'
        
        return False, None
    
    def _calculate_positions_value(self, positions, daily_options):
        """Calculate total value of open positions"""
        total_value = 0
        
        for position in positions:
            option_data = daily_options[
                (daily_options['strike'] == position['strike']) &
                (daily_options['expiration'] == position['expiration'])
            ]
            
            if not option_data.empty:
                current_price = option_data.iloc[0]['close']
                
                if position['type'] == 'leap':
                    # LEAP is long position (asset)
                    total_value += current_price * 100
                else:
                    # Short call is short position (liability)
                    total_value -= current_price * 100
        
        return total_value
    
    def _calculate_results(self, equity_curve, trades, params, verbose):
        """Calculate backtest performance metrics"""
        if not equity_curve:
            return {'error': 'No equity curve data'}
        
        eq_df = pd.DataFrame(equity_curve)
        
        # Basic metrics
        initial_value = eq_df['total_value'].iloc[0]
        final_value = eq_df['total_value'].iloc[-1]
        total_return = (final_value - initial_value) / initial_value * 100
        
        # Daily returns for Sharpe ratio
        eq_df['daily_returns'] = eq_df['total_value'].pct_change()
        returns_series = eq_df['daily_returns'].dropna()
        
        if len(returns_series) > 1:
            sharpe_ratio = returns_series.mean() / returns_series.std() * np.sqrt(252)
        else:
            sharpe_ratio = 0
        
        # Drawdown
        eq_df['peak'] = eq_df['total_value'].expanding().max()
        eq_df['drawdown'] = (eq_df['total_value'] - eq_df['peak']) / eq_df['peak'] * 100
        max_drawdown = eq_df['drawdown'].min()
        
        # SPY comparison
        spy_initial = eq_df['spy_price'].iloc[0]
        spy_final = eq_df['spy_price'].iloc[-1]
        spy_return = (spy_final - spy_initial) / spy_initial * 100
        
        # Trade statistics
        trades_df = pd.DataFrame(trades) if trades else pd.DataFrame()
        win_rate = 0
        avg_trade = 0
        if not trades_df.empty and 'pnl' in trades_df.columns:
            profit_trades = trades_df[trades_df['pnl'] > 0]
            win_rate = len(profit_trades) / len(trades_df) * 100
            avg_trade = trades_df['pnl'].mean()
        
        results = {
            'initial_value': initial_value,
            'final_value': final_value,
            'total_return': total_return,
            'sharpe_ratio': sharpe_ratio,
            'max_drawdown': max_drawdown,
            'spy_return': spy_return,
            'excess_return': total_return - spy_return,
            'total_trades': len(trades),
            'win_rate': win_rate,
            'avg_trade': avg_trade,
            'trading_days': len(equity_curve)
        }
        
        if verbose:
            print(f"\n📊 BACKTEST RESULTS:")
            print(f"  Total Return: {total_return:.2f}%")
            print(f"  Sharpe Ratio: {sharpe_ratio:.2f}")
            print(f"  Max Drawdown: {max_drawdown:.2f}%") 
            print(f"  Win Rate: {win_rate:.1f}%")
            print(f"  Total Trades: {len(trades)}")
            print(f"  SPY Return: {spy_return:.2f}%")
            print(f"  Excess Return: {total_return - spy_return:.2f}%")
        
        return results

In [ ]:
# Initialize with message
print("🎯 PMCC Optimization Framework Ready!")
print("📋 Next: Load data and run baseline backtest")

In [ ]:
"""
Load Data and Initialize Optimizer
"""

print("📊 Loading SPY options data...")
# Load comprehensive options data
options_data = load_data(
    'data/spy_options/', 
    start_date='2023-01-01', 
    end_date='2024-12-31'
)

if options_data is None:
    print("❌ Failed to load options data")
else:
    print(f"✅ Loaded {len(options_data):,} option records")
    print(f"📅 Date range: {options_data['date'].min()} to {options_data['date'].max()}")

# Create SPY price data from options
spy_data = options_data.groupby('date')['underlying_price'].first().reset_index()
spy_data.columns = ['date', 'close']
print(f"📈 Created SPY price series: {len(spy_data)} days")

In [ ]:
"""
Initialize Optimizer and Run Baseline Test
"""

# Initialize the optimizer
optimizer = PMCCOptimizer(
    data=options_data,
    spy_data=spy_data,
    initial_capital=10000
)

print("🚀 Running baseline PMCC backtest...")

# Run baseline backtest (no filters enabled)
baseline_results = optimizer.run_backtest(
    params=None,  # Use default parameters
    start_date='2023-01-01',
    end_date='2024-12-31',
    verbose=True
)

if baseline_results is None:
    print("❌ Baseline backtest failed")
else:
    print("\n" + "="*50)
    print("📊 BASELINE PMCC RESULTS")
    print("="*50)
    print(f"Total Return:     {baseline_results['total_return']:.2f}%")
    print(f"Sharpe Ratio:     {baseline_results['sharpe_ratio']:.2f}")
    print(f"Max Drawdown:     {baseline_results['max_drawdown']:.2f}%")
    print(f"Win Rate:         {baseline_results['win_rate']:.1f}%")
    print(f"Total Trades:     {baseline_results['total_trades']}")
    print(f"SPY Return:       {baseline_results['spy_return']:.2f}%")
    print(f"Excess Return:    {baseline_results['excess_return']:.2f}%")
    print(f"Avg Trade P&L:    ${baseline_results['avg_trade']:.2f}")

# Store baseline for comparison
baseline_sharpe = baseline_results['sharpe_ratio']
baseline_return = baseline_results['total_return']

In [ ]:
"""
Market Timing Filter Tests
"""

print("\n🎯 TESTING MARKET TIMING FILTERS")
print("="*40)

# Test 1: VIX Filter (using volatility proxy)
print("\n1️⃣ VIX Filter Test")
vix_params = optimizer.default_params.copy()
vix_params.update({
    'use_vix_filter': True,
    'vix_entry_max': 25,
    'vix_exit_min': 35
})

vix_results = optimizer.run_backtest(
    params=vix_params,
    start_date='2023-01-01',
    end_date='2024-12-31',
    verbose=False
)

if vix_results:
    print(f"VIX Filter - Sharpe: {vix_results['sharpe_ratio']:.2f} (vs {baseline_sharpe:.2f})")
    print(f"VIX Filter - Return: {vix_results['total_return']:.2f}% (vs {baseline_return:.2f}%)")
    print(f"VIX Filter - Trades: {vix_results['total_trades']} (vs {baseline_results['total_trades']})")
    vix_improvement = vix_results['sharpe_ratio'] - baseline_sharpe
    print(f"Sharpe Improvement: {vix_improvement:+.3f}")

# Test 2: Trend Filter (SMA 50)
print("\n2️⃣ Trend Filter Test (50-day SMA)")
trend_params = optimizer.default_params.copy()
trend_params.update({
    'use_trend_filter': True,
    'sma_period': 50,
    'require_uptrend': True
})

trend_results = optimizer.run_backtest(
    params=trend_params,
    start_date='2023-01-01',
    end_date='2024-12-31',
    verbose=False
)

if trend_results:
    print(f"Trend Filter - Sharpe: {trend_results['sharpe_ratio']:.2f} (vs {baseline_sharpe:.2f})")
    print(f"Trend Filter - Return: {trend_results['total_return']:.2f}% (vs {baseline_return:.2f}%)")
    print(f"Trend Filter - Trades: {trend_results['total_trades']} (vs {baseline_results['total_trades']})")
    trend_improvement = trend_results['sharpe_ratio'] - baseline_sharpe
    print(f"Sharpe Improvement: {trend_improvement:+.3f}")

# Test 3: RSI Filter
print("\n3️⃣ RSI Filter Test (30-70 range)")
rsi_params = optimizer.default_params.copy()
rsi_params.update({
    'use_rsi_filter': True,
    'rsi_period': 14,
    'rsi_min': 30,
    'rsi_max': 70
})

rsi_results = optimizer.run_backtest(
    params=rsi_params,
    start_date='2023-01-01',
    end_date='2024-12-31',
    verbose=False
)

if rsi_results:
    print(f"RSI Filter - Sharpe: {rsi_results['sharpe_ratio']:.2f} (vs {baseline_sharpe:.2f})")
    print(f"RSI Filter - Return: {rsi_results['total_return']:.2f}% (vs {baseline_return:.2f}%)")
    print(f"RSI Filter - Trades: {rsi_results['total_trades']} (vs {baseline_results['total_trades']})")
    rsi_improvement = rsi_results['sharpe_ratio'] - baseline_sharpe
    print(f"Sharpe Improvement: {rsi_improvement:+.3f}")

# Test 4: Combined Filters
print("\n4️⃣ Combined Filter Test (VIX + Trend)")
combined_params = optimizer.default_params.copy()
combined_params.update({
    'use_vix_filter': True,
    'vix_entry_max': 25,
    'vix_exit_min': 35,
    'use_trend_filter': True,
    'sma_period': 50,
    'require_uptrend': True
})

combined_results = optimizer.run_backtest(
    params=combined_params,
    start_date='2023-01-01',
    end_date='2024-12-31',
    verbose=False
)

if combined_results:
    print(f"Combined - Sharpe: {combined_results['sharpe_ratio']:.2f} (vs {baseline_sharpe:.2f})")
    print(f"Combined - Return: {combined_results['total_return']:.2f}% (vs {baseline_return:.2f}%)")
    print(f"Combined - Trades: {combined_results['total_trades']} (vs {baseline_results['total_trades']})")
    combined_improvement = combined_results['sharpe_ratio'] - baseline_sharpe
    print(f"Sharpe Improvement: {combined_improvement:+.3f}")

print("\n🎯 Market Timing Filter Summary:")
if vix_results and trend_results and rsi_results and combined_results:
    filter_results = [
        ("Baseline", baseline_sharpe),
        ("VIX Filter", vix_results['sharpe_ratio']),
        ("Trend Filter", trend_results['sharpe_ratio']),
        ("RSI Filter", rsi_results['sharpe_ratio']),
        ("VIX + Trend", combined_results['sharpe_ratio'])
    ]
    
    # Sort by Sharpe ratio
    filter_results.sort(key=lambda x: x[1], reverse=True)
    
    for i, (name, sharpe) in enumerate(filter_results):
        symbol = "🥇" if i == 0 else "🥈" if i == 1 else "🥉" if i == 2 else "📊"
        improvement = sharpe - baseline_sharpe
        print(f"  {symbol} {name}: {sharpe:.3f} ({improvement:+.3f})")

In [ ]:
"""
Delta Target Optimization Tests  
"""

print("\n🎯 TESTING DELTA TARGET OPTIMIZATION")
print("="*40)

# Test different LEAP delta ranges
leap_delta_tests = [
    ("Conservative (0.65-0.75)", 0.65, 0.75),
    ("Baseline (0.70-0.85)", 0.70, 0.85),
    ("Aggressive (0.75-0.90)", 0.75, 0.90)
]

print("\n📈 LEAP Delta Optimization:")
leap_results = []
for name, min_delta, max_delta in leap_delta_tests:
    params = optimizer.default_params.copy()
    params.update({
        'leap_delta_min': min_delta,
        'leap_delta_max': max_delta
    })
    
    results = optimizer.run_backtest(params=params, start_date='2023-01-01', end_date='2024-12-31')
    if results:
        leap_results.append((name, results['sharpe_ratio'], results['total_return']))
        improvement = results['sharpe_ratio'] - baseline_sharpe
        print(f"  {name}: Sharpe {results['sharpe_ratio']:.3f} ({improvement:+.3f})")

# Test different short call delta ranges  
short_delta_tests = [
    ("Conservative (0.15-0.25)", 0.15, 0.25),
    ("Baseline (0.20-0.30)", 0.20, 0.30),
    ("Aggressive (0.25-0.35)", 0.25, 0.35)
]

print("\n📉 Short Call Delta Optimization:")
short_results = []
for name, min_delta, max_delta in short_delta_tests:
    params = optimizer.default_params.copy()
    params.update({
        'short_delta_min': min_delta,
        'short_delta_max': max_delta
    })
    
    results = optimizer.run_backtest(params=params, start_date='2023-01-01', end_date='2024-12-31')
    if results:
        short_results.append((name, results['sharpe_ratio'], results['total_return']))
        improvement = results['sharpe_ratio'] - baseline_sharpe
        print(f"  {name}: Sharpe {results['sharpe_ratio']:.3f} ({improvement:+.3f})")

In [ ]:
print("\n🎯 OPTIMIZATION SUMMARY")
print("="*50)
print(f"🏁 Baseline Sharpe Ratio: {baseline_sharpe:.3f}")
print("\n📊 Best Improvements Found:")

# Collect all test results
all_results = []
if vix_results:
    all_results.append(("VIX Filter", vix_results['sharpe_ratio']))
if trend_results:
    all_results.append(("Trend Filter", trend_results['sharpe_ratio']))
if combined_results:
    all_results.append(("VIX + Trend", combined_results['sharpe_ratio']))

for name, sharpe, _ in leap_results:
    all_results.append((f"LEAP {name}", sharpe))

for name, sharpe, _ in short_results:
    all_results.append((f"Short {name}", sharpe))

# Find top improvements
if all_results:
    all_results.sort(key=lambda x: x[1], reverse=True)
    
    print("Top 5 Configurations:")
    for i, (name, sharpe) in enumerate(all_results[:5]):
        improvement = sharpe - baseline_sharpe
        symbol = "🥇" if i == 0 else "🥈" if i == 1 else "🥉" if i == 2 else "⭐"
        print(f"  {symbol} {name}: {sharpe:.3f} ({improvement:+.3f})")
        
    best_improvement = all_results[0][1] - baseline_sharpe
    best_config = all_results[0][0]
    
    if best_improvement > 0:
        print(f"\n✅ Best Enhancement: {best_config}")
        print(f"📈 Sharpe Improvement: {best_improvement:.3f} ({best_improvement/baseline_sharpe*100:.1f}%)")
    else:
        print(f"\n⚠️ No improvements found - baseline parameters are optimal for this period")

In [ ]:
"""
Advanced Optimization: Greeks-Based Management
"""

print("\n🎯 TESTING GREEKS-BASED OPTIMIZATION")
print("="*40)

# Add Greeks management methods to optimizer
def enhanced_exit_conditions(self, position, current_price, days_to_expiry, params, current_option=None):
    """Enhanced exit conditions with Greeks-based rules"""
    
    # Get current Greeks if available
    current_delta = current_option.get('delta', 0) if current_option is not None else 0
    current_vega = current_option.get('vega', 0) if current_option is not None else 0
    current_theta = current_option.get('theta', 0) if current_option is not None else 0
    
    # Standard exit conditions first
    should_exit, exit_reason = self._check_exit_conditions(position, current_price, days_to_expiry, params)
    if should_exit:
        return should_exit, exit_reason
    
    # Greeks-based exits
    if position['type'] == 'leap':
        # LEAP Greeks management
        if 'use_delta_management' in params and params['use_delta_management']:
            # Roll LEAP if delta drops too low (loses leverage)
            if current_delta < params.get('leap_min_delta', 0.60):
                return True, f'delta_too_low_{current_delta:.2f}'
            
            # Roll LEAP if delta too high (assignment risk on underlying)
            if current_delta > params.get('leap_max_delta', 0.95):
                return True, f'delta_too_high_{current_delta:.2f}'
        
        if 'use_vega_management' in params and params['use_vega_management']:
            # Roll LEAP if vega exposure too high in low vol environment
            if abs(current_vega) > params.get('leap_max_vega', 0.50):
                return True, f'vega_too_high_{abs(current_vega):.2f}'
    
    else:
        # Short call Greeks management
        if 'use_delta_management' in params and params['use_delta_management']:
            # Buy back short if delta gets too high (assignment risk)
            if current_delta > params.get('short_max_delta', 0.60):
                return True, f'short_delta_risk_{current_delta:.2f}'
        
        if 'use_theta_optimization' in params and params['use_theta_optimization']:
            # Keep position if theta decay is accelerating (last 30 days)
            if days_to_expiry <= 30 and abs(current_theta) > params.get('min_theta_keep', 0.05):
                # Don't exit early if theta is working for us
                return False, None
    
    return False, None

# Monkey patch the enhanced method
PMCCOptimizer._check_exit_conditions_enhanced = enhanced_exit_conditions

# Test Greeks-based management
print("\n1️⃣ Delta Management Test")
delta_params = optimizer.default_params.copy()
delta_params.update({
    'use_delta_management': True,
    'leap_min_delta': 0.60,
    'leap_max_delta': 0.95,
    'short_max_delta': 0.60
})

# Temporarily patch the exit condition method
original_exit_method = optimizer._check_exit_conditions
optimizer._check_exit_conditions = lambda pos, price, dte, params: enhanced_exit_conditions(optimizer, pos, price, dte, params)

delta_results = optimizer.run_backtest(
    params=delta_params,
    start_date='2023-01-01',
    end_date='2024-12-31',
    verbose=False
)

# Restore original method
optimizer._check_exit_conditions = original_exit_method

if delta_results:
    print(f"Delta Mgmt - Sharpe: {delta_results['sharpe_ratio']:.2f} (vs {baseline_sharpe:.2f})")
    print(f"Delta Mgmt - Return: {delta_results['total_return']:.2f}% (vs {baseline_return:.2f}%)")
    delta_improvement = delta_results['sharpe_ratio'] - baseline_sharpe
    print(f"Sharpe Improvement: {delta_improvement:+.3f}")

# Test 2: Theta Optimization
print("\n2️⃣ Theta Optimization Test")
theta_params = optimizer.default_params.copy()
theta_params.update({
    'use_theta_optimization': True,
    'min_theta_keep': 0.05,
    'roll_dte_threshold': 14  # Hold longer for theta decay
})

theta_results = optimizer.run_backtest(
    params=theta_params,
    start_date='2023-01-01',
    end_date='2024-12-31',
    verbose=False
)

if theta_results:
    print(f"Theta Opt - Sharpe: {theta_results['sharpe_ratio']:.2f} (vs {baseline_sharpe:.2f})")
    print(f"Theta Opt - Return: {theta_results['total_return']:.2f}% (vs {baseline_return:.2f}%)")
    theta_improvement = theta_results['sharpe_ratio'] - baseline_sharpe
    print(f"Sharpe Improvement: {theta_improvement:+.3f}")

In [ ]:
"""
IV Rank-Based Optimization
"""

print("\n🎯 TESTING IV RANK OPTIMIZATION")  
print("="*40)

# Add IV rank calculation to SPY data
def calculate_iv_rank(data, window=252):
    """Calculate IV rank for the data"""
    if 'vol_20' not in data.columns:
        return data
    
    data = data.copy()
    data['iv_rank'] = data['vol_20'].rolling(window).rank(pct=True)
    return data

# Update SPY data with IV rank
optimizer.spy_data = calculate_iv_rank(optimizer.spy_data)

# Test IV rank filters
print("\n1️⃣ IV Entry Filter Test")
iv_entry_params = optimizer.default_params.copy()
iv_entry_params.update({
    'use_iv_filter': True,
    'iv_rank_min': 0.30,  # Only sell calls when vol > 30th percentile
    'iv_rank_max': 1.00   # No upper limit for entry
})

iv_entry_results = optimizer.run_backtest(
    params=iv_entry_params,
    start_date='2023-01-01',
    end_date='2024-12-31',
    verbose=False
)

if iv_entry_results:
    print(f"IV Entry - Sharpe: {iv_entry_results['sharpe_ratio']:.2f} (vs {baseline_sharpe:.2f})")
    print(f"IV Entry - Return: {iv_entry_results['total_return']:.2f}% (vs {baseline_return:.2f}%)")
    iv_entry_improvement = iv_entry_results['sharpe_ratio'] - baseline_sharpe
    print(f"Sharpe Improvement: {iv_entry_improvement:+.3f}")

print("\n2️⃣ IV Exit Filter Test")
iv_exit_params = optimizer.default_params.copy()
iv_exit_params.update({
    'use_iv_filter': True,
    'iv_rank_min': 0.10,  # Exit positions when vol < 10th percentile  
    'iv_rank_max': 0.90   # Enter LEAPs when vol < 90th percentile
})

iv_exit_results = optimizer.run_backtest(
    params=iv_exit_params,
    start_date='2023-01-01',
    end_date='2024-12-31',
    verbose=False
)

if iv_exit_results:
    print(f"IV Exit - Sharpe: {iv_exit_results['sharpe_ratio']:.2f} (vs {baseline_sharpe:.2f})")
    print(f"IV Exit - Return: {iv_exit_results['total_return']:.2f}% (vs {baseline_return:.2f}%)")
    iv_exit_improvement = iv_exit_results['sharpe_ratio'] - baseline_sharpe
    print(f"Sharpe Improvement: {iv_exit_improvement:+.3f}")

In [ ]:
"""
Collar Modifications (PMCC + Protective Put)
"""

print("\n🎯 TESTING COLLAR MODIFICATIONS")
print("="*40)

# Add collar functionality to the optimizer
def add_protective_put(self, daily_options, current_date, cash, params, leap_position):
    """Add protective put to PMCC position"""
    if not params.get('use_collar', False):
        return None, cash, []
    
    # Find suitable protective put
    puts = daily_options[daily_options['right'] == 'P'].copy()
    
    # Filter by DTE (match short call)
    puts['dte'] = (pd.to_datetime(puts['expiration']) - pd.to_datetime(current_date)).dt.days
    put_candidates = puts[
        (puts['dte'] >= params.get('collar_dte_min', 30)) &
        (puts['dte'] <= params.get('collar_dte_max', 45))
    ]
    
    if put_candidates.empty:
        return None, cash, []
    
    # Filter for OTM puts (protection level)
    underlying_price = daily_options['underlying_price'].iloc[0]
    protection_level = params.get('collar_protection_pct', 0.10)  # 10% protection
    target_strike = underlying_price * (1 - protection_level)
    
    put_candidates = put_candidates[
        put_candidates['strike'] <= target_strike
    ]
    
    # Basic liquidity
    put_candidates = put_candidates[
        (put_candidates['bid'] > 0) &
        (put_candidates['volume'] > 0)
    ]
    
    if put_candidates.empty:
        return None, cash, []
    
    # Select closest to target strike
    put_candidates['strike_diff'] = abs(put_candidates['strike'] - target_strike)
    best_put = put_candidates.nsmallest(1, 'strike_diff').iloc[0]
    
    # Buy protective put
    put_cost = best_put['close'] * 100 + params['commission_per_contract']
    put_cost *= (1 + params['slippage_pct'])
    
    if put_cost > cash * 0.1:  # Don't spend more than 10% on protection
        return None, cash, []
    
    cash -= put_cost
    
    put_position = {
        'type': 'protective_put',
        'entry_date': current_date,
        'strike': best_put['strike'],
        'expiration': best_put['expiration'],
        'entry_price': best_put['close'],
        'cost': put_cost,
        'contracts': 1,
        'leap_strike': leap_position['strike'],
        'leap_expiration': leap_position['expiration']
    }
    
    trade = {
        'date': current_date,
        'action': 'buy_protective_put',
        'strike': best_put['strike'],
        'expiration': best_put['expiration'],
        'price': best_put['close'],
        'cost': put_cost,
        'pnl': 0,
        'reason': 'collar_protection'
    }
    
    return put_position, cash, [trade]

# Monkey patch collar functionality
PMCCOptimizer.add_protective_put = add_protective_put

# Test collar modifications
print("\n1️⃣ Basic Collar Test (10% Protection)")
collar_params = optimizer.default_params.copy()
collar_params.update({
    'use_collar': True,
    'collar_protection_pct': 0.10,  # 10% downside protection
    'collar_dte_min': 30,
    'collar_dte_max': 45
})

collar_results = optimizer.run_backtest(
    params=collar_params,
    start_date='2023-01-01',
    end_date='2024-12-31',
    verbose=False
)

if collar_results:
    print(f"Collar 10% - Sharpe: {collar_results['sharpe_ratio']:.2f} (vs {baseline_sharpe:.2f})")
    print(f"Collar 10% - Return: {collar_results['total_return']:.2f}% (vs {baseline_return:.2f}%)")
    collar_improvement = collar_results['sharpe_ratio'] - baseline_sharpe
    print(f"Sharpe Improvement: {collar_improvement:+.3f}")

print("\n2️⃣ Tight Collar Test (5% Protection)")
tight_collar_params = optimizer.default_params.copy()
tight_collar_params.update({
    'use_collar': True,
    'collar_protection_pct': 0.05,  # 5% protection
    'collar_dte_min': 30,
    'collar_dte_max': 45
})

tight_collar_results = optimizer.run_backtest(
    params=tight_collar_params,
    start_date='2023-01-01',
    end_date='2024-12-31',
    verbose=False
)

if tight_collar_results:
    print(f"Collar 5% - Sharpe: {tight_collar_results['sharpe_ratio']:.2f} (vs {baseline_sharpe:.2f})")
    print(f"Collar 5% - Return: {tight_collar_results['total_return']:.2f}% (vs {baseline_return:.2f}%)")
    tight_collar_improvement = tight_collar_results['sharpe_ratio'] - baseline_sharpe
    print(f"Sharpe Improvement: {tight_collar_improvement:+.3f}")

In [ ]:
"""
Comprehensive Parameter Sweep Analysis
"""

print("\n🎯 COMPREHENSIVE PARAMETER SWEEP")
print("="*50)

# Collect all results for final analysis
all_optimization_results = [
    ("Baseline", baseline_sharpe, baseline_return, baseline_results['max_drawdown'])
]

# Add all test results
test_results = [
    ("VIX Filter", vix_results),
    ("Trend Filter", trend_results), 
    ("RSI Filter", rsi_results),
    ("VIX + Trend", combined_results),
    ("Delta Management", delta_results),
    ("Theta Optimization", theta_results),
    ("IV Entry Filter", iv_entry_results),
    ("IV Exit Filter", iv_exit_results),
    ("Collar 10%", collar_results),
    ("Collar 5%", tight_collar_results)
]

for name, results in test_results:
    if results:
        all_optimization_results.append((
            name, 
            results['sharpe_ratio'], 
            results['total_return'],
            results['max_drawdown']
        ))

# Sort by Sharpe ratio
all_optimization_results.sort(key=lambda x: x[1], reverse=True)

print("🏆 FINAL OPTIMIZATION RANKINGS:")
print("="*50)
for i, (name, sharpe, ret, dd) in enumerate(all_optimization_results):
    improvement = sharpe - baseline_sharpe
    symbol = "🥇" if i == 0 else "🥈" if i == 1 else "🥉" if i == 2 else f"{i+1:2d}."
    
    print(f"{symbol} {name:<20} | Sharpe: {sharpe:5.3f} ({improvement:+.3f}) | Return: {ret:6.1f}% | DD: {dd:5.1f}%")

# Best configuration summary
if len(all_optimization_results) > 0:
    best_name, best_sharpe, best_return, best_dd = all_optimization_results[0]
    
    print(f"\n🎉 BEST CONFIGURATION: {best_name}")
    print("="*50)
    print(f"📈 Sharpe Ratio:    {best_sharpe:.3f} (vs {baseline_sharpe:.3f} baseline)")
    print(f"💰 Total Return:    {best_return:.1f}% (vs {baseline_return:.1f}% baseline)")
    print(f"📉 Max Drawdown:    {best_dd:.1f}% (vs {baseline_results['max_drawdown']:.1f}% baseline)")
    
    sharpe_improvement = best_sharpe - baseline_sharpe
    return_improvement = best_return - baseline_return
    
    print(f"\n✨ IMPROVEMENTS:")
    print(f"   Sharpe: +{sharpe_improvement:.3f} ({sharpe_improvement/baseline_sharpe*100:+.1f}%)")
    print(f"   Return: +{return_improvement:.1f}%")
    
    if sharpe_improvement > 0.1:
        print(f"\n🚀 SIGNIFICANT IMPROVEMENT FOUND!")
        print(f"   The {best_name} configuration shows meaningful Sharpe ratio enhancement")
        print(f"   Consider implementing this optimization for live trading")
    elif sharpe_improvement > 0.05:
        print(f"\n📊 MODERATE IMPROVEMENT FOUND")
        print(f"   The {best_name} shows modest but consistent improvement")
    else:
        print(f"\n📋 BASELINE REMAINS OPTIMAL")
        print(f"   Current parameters appear well-calibrated for this period")

print(f"\n✅ PMCC Optimization Research Complete!")
print(f"🎯 Framework successfully tested {len(test_results)} enhancement strategies")
print(f"📊 Ready for production implementation or further research")